# <span style="color:blue; font-size:16;">SGF 2020 Super Demo SASPy</span>
# <span style="color:blue; font-size:16;">What's new in SASPy since SGF 2019?</span>

## Important links
Repo: https://github.com/sassoftware/saspy
        
Doc: https://sassoftware.github.io/saspy/index.html
        
Examples: https://github.com/sassoftware/saspy-examples
        

In [1]:
import saspy
saspy

<module 'saspy' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\saspy\\__init__.py'>

### The last feature in saspy before SGF 2019 was support for Zeppelin Notebooks

# Since SGF 2019, there have been 13 releases of SASPy
## there have been fixes for issues, enhancements to functionslity and new features 

# V3.0.0

## New Access Method named HTTP
## it provides access to SAS in a Viya install via the Compute Service

### Let's see what a Compute Service connection looks like

In [2]:
sas_viya = saspy.SASsession(cfgname='itviya', results='HTML')
sas_viya

SAS server started using Context Data Mining compute context with SESSION_ID=88f5acbe-6c23-476e-99a5-dbb757ed0a21-ses0000


Access Method         = HTTP
SAS Config name       = itviya
SAS Config file       = C:\ProgramData\Anaconda3\lib\site-packages\saspy\sascfg_personal.py
WORK Path             = /sastmp/SAS_work30C800006B43_itviyap101.itviya.sashq-d.openstack.sas.com/
SAS Version           = V.03.05M0P11062019
SASPy Version         = 3.2.0
Teach me SAS          = False
Batch                 = False
Results               = HTML
SAS Session Encoding  = utf-8
Python Encoding value = utf_8
SAS process Pid value = 27459


# V3.0.1

## New Access Method named COM -  This was USER CONTRIBUTED!
## it uses the IOM COM client to provide access to workspace servers.
## It is Windows client only, and uses the same interface as EG

### And how about COM?

In [3]:
sas_com = saspy.SASsession(cfgname='iomcom_loc', results='HTML')
sas_com

SAS Connection established. Workspace UniqueIdentifier is ECC25E22-423A-4875-A051-8E02C770EF9B

No encoding value provided. Will try to determine the correct encoding.
Setting encoding to cp1252 based upon the SAS session encoding value of wlatin1.



Access Method         = COM
SAS Config name       = iomcom_loc
SAS Config file       = C:\ProgramData\Anaconda3\lib\site-packages\saspy\sascfg_personal.py
WORK Path             = C:\Users\sastpw\AppData\Local\Temp\SAS Temporary Files\_TD27776_d10a626_\Prc2\
SAS Version           = 9.04.01M5P09132017
SASPy Version         = 3.2.0
Teach me SAS          = False
Batch                 = False
Results               = HTML
SAS Session Encoding  = wlatin1
Python Encoding value = cp1252
SAS process Pid value = 27776


## Another addition in V3.0.1 are jars to support CORBA

## Create a local SASsession object to use for the rest of this

In [4]:
sas = saspy.SASsession(cfgname='winlocal', results='HTML')
sas

SAS Connection established. Subprocess id is 19452



Access Method         = IOM
SAS Config name       = winlocal
SAS Config file       = C:\ProgramData\Anaconda3\lib\site-packages\saspy\sascfg_personal.py
WORK Path             = C:\Users\sastpw\AppData\Local\Temp\SAS Temporary Files\_TD25056_d10a626_\Prc2\
SAS Version           = 9.04.01M5P09132017
SASPy Version         = 3.2.0
Teach me SAS          = False
Batch                 = False
Results               = HTML
SAS Session Encoding  = wlatin1
Python Encoding value = cp1252
SAS process Pid value = 25056


## This local session uses IOM in local mode. 
## IOM uses the Java IOM client, which requires CORBA support
## After Java 9, CORBA is no longer part of Java. That broke IOM. 
## But, new jars were put together to add in this support. 
## These jars are included in SASPy so it works with newer Java  # V3.0.1 

<pre>
# Java 10+ no longer provides CORBA (9 has it but doesn't load it by default). These jars provide the CORBA support
# that the IOM client needs. Add these to the classpath to work with the latest Java releases.
# These can even be in the classpath for Jave (7 or 8) that don't need them, with no problem.
cp += ";C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\saspy\\java\\thirdparty\\glassfish-corba-internal-api.jar"
cp += ";C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\saspy\\java\\thirdparty\\glassfish-corba-omgapi.jar"
cp += ";C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\saspy\\java\\thirdparty\\glassfish-corba-orb.jar"
cp += ";C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\saspy\\java\\thirdparty\\pfl-basic.jar"
cp += ";C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\saspy\\java\\thirdparty\\pfl-tf.jar"
</pre>

# V3.1.5

## enhanced support for importing character data with embedded newlines via df2sd()

### Let's look at the newline problem

In [5]:
import pandas as pd
rows = [['Hi\nthere\nhow\nare\nyou?', 4]]
df = pd.DataFrame.from_records(rows, columns=['paragraph','number'])
df

,paragraph,number
0,Hi\nthere\nhow\nare\nyou?,4


In [6]:
print(df['paragraph'][0])

Hi
there
how
are
you?


In [7]:
sd = sas.df2sd(df)
sd

Libref  = WORK
Table   = _df
Dsopts  = {}
Results = HTML

In [8]:
sd.head()

## <span style="color:red; font-size:16;">Wait, what? What's up with that?</span>
#### See this in the log
<pre>ERROR: INPUT statement exceeded record length. INFILE CARDS OPTION STOPOVER specified.
RULE:      ----+----1----+----2----+----3----+----4----+----5----+----6----+----7----+----8----+----9----+----0                     
29         Hi
'paragraph'n=Hi 'number'n=. _ERROR_=1 _INFILE_=Hi _N_=2
NOTE: DATA statement used (Total process time):
      real time           0.01 seconds
      cpu time            0.01 seconds
      
NOTE: The SAS System stopped processing this step because of errors.
WARNING: The data set WORK._DF may be incomplete.  When this step was stopped there were 0 observations and 2 variables.
</pre>

In [9]:
sd = sas.df2sd(df)
print(sas.saslog())

1                                                          The SAS System                             09:42 Thursday, March 12, 2020

NOTE: Copyright (c) 2016 by SAS Institute Inc., Cary, NC, USA. 
NOTE: SAS (r) Proprietary Software 9.4 (TS1M5) 
      Licensed to Weber Tom-1347, Site 70068130.
NOTE: This session is executing on the X64_10PRO  platform.



NOTE: Updated analytical products:
      
      SAS/STAT 14.3
      SAS/ETS 14.3
      SAS/OR 14.3
      SAS/QC 14.3

NOTE: Additional host information:

 X64_10PRO WIN 10.0.16299  Workstation

NOTE: SAS Initialization used (Total process time):
      real time           0.00 seconds
      cpu time            0.01 seconds
      
1          ;*';*";*/;
2          options svgtitle='svgtitle'; options validvarname=any validmemname=extend pagesize=max nosyntaxcheck; ods graphics on;
3          
4          ;*';*";*/;
5          %put E3969440A681A2408885998500000001;
E3969440A681A2408885998500000001
6          
2                           

### We can import this by setting embedded_newlines=True
### saspy will generate the extra code to convert newlines to/from
### other hex values in Python/SAS so this can import correctly
### This requires extra processesing, which is why it'snot the default

In [10]:
sd = sas.df2sd(df, embedded_newlines=True, results='text')
sd

Libref  = WORK
Table   = _df
Dsopts  = {}
Results = text

In [11]:
sd.head()


                                                           The SAS System                         09:42 Thursday, March 12, 2020   1

                                               Obs          paragraph          number

                                                1     Hi
there
how
are
you?       4  



In [12]:
# we can see when we bring it back into a dataframe that the newlines really did round trip
df2 = sd.to_df()
df2

,paragraph,number
0,Hi\nthere\nhow\nare\nyou?,4


In [13]:
print(df2['paragraph'][0])

Hi
there
how
are
you?


## there's a new method for running proc mortgage: sil()  AKA: simple_interest_loan(()

### So, what's SimpleInterestLoan() ?

In [14]:
sas.simple_interest_loan(life=12,   rate=3.5, amount=1000, payment=None)
sas.simple_interest_loan(life=None, rate=3.5, amount=1000, payment=100)

## and a new method for adding variables to a SAS Data Set: add_vars()
### new columns ar calculated using existing variables and/or constants

In [15]:
cars = sas.sasdata('cars', 'sashelp')
wkcars = sas.sasdata('cars')
cars.add_vars({'PW_ratio': 'weight / horsepower', 'Overhang' : 'length - wheelbase'}, wkcars)
wkcars.head()

Table WORK.cars does not exist. This SASdata object will not be useful until the data set is created.
34                                                         The SAS System                             09:42 Thursday, March 12, 2020

288        
289        data WORK.'cars'n ; set sashelp.'cars'n ;
290        PW_ratio = weight / horsepower;
291        Overhang = length - wheelbase;
292        ; run;

NOTE: There were 428 observations read from the data set SASHELP.CARS.
NOTE: The data set WORK.CARS has 428 observations and 17 variables.
NOTE: DATA statement used (Total process time):
      real time           0.00 seconds
      cpu time            0.01 seconds
      

293        
294        


Obs,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length,PW_ratio,Overhang
1,Acura,MDX,SUV,Asia,All,"$36,945","$33,337",3.5,6,265,17,23,4451,106,189,16.7962,83
2,Acura,RSX Type S 2dr,Sedan,Asia,Front,"$23,820","$21,761",2.0,4,200,24,31,2778,101,172,13.8900,71
3,Acura,TSX 4dr,Sedan,Asia,Front,"$26,990","$24,647",2.4,4,200,22,29,3230,105,183,16.1500,78
4,Acura,TL 4dr,Sedan,Asia,Front,"$33,195","$30,299",3.2,6,270,20,28,3575,108,186,13.2407,78
5,Acura,3.5 RL 4dr,Sedan,Asia,Front,"$43,755","$39,014",3.5,6,225,18,24,3880,115,197,17.2444,82


# V3.1.6 

## support for having sascfg_personal.py in home Directory was contributed by a user

## New search path for config file (backward compatible if you don't use new ~/.config/saspy dir)
### 1) saspy install directory (same place as the sascfg.py examples file in to repo)
### 2) current dir (whatever that is; wherever you started python)
### 3) ~/.config/saspy - the new directory being added in 3.1.6 (that works on windows too: think %HOMEPATH%\.config\saspy)
### 4) the rest of the directories in the python path
## if not found, then sascfg.py in the saspy repo (only to support way, way, way back compatibility - don’t use this)

In [16]:
saspy.list_configs()

['C:\\ProgramData\\Anaconda3\\lib\\site-packages\\saspy\\sascfg_personal.py',
 'C:\\Users\\sastpw/.config/saspy\\sascfg_personal.py']

# V3.1.7

## New DISK method for sd2df

In [17]:
df = sd.to_df_DISK() # sd.to_df(..., method='disk')    sas.sd2df_DISK()   sas.sd2df(..., method='DISK') 
df

,paragraph,number
0,Hi\nthere\nhow\nare\nyou?,4.0


## and outtype= on symget():   symget('variable', outtype=[int, float, str])

In [18]:
x = sas.symget('syserr')
x

0

In [19]:
type(x)

int

In [20]:
x = sas.symget('syserr', outtype=str())
x

'0'

In [21]:
type(x)

str

In [22]:
x = sas.symget('syserr', outtype=float())
x

0.0

In [23]:
type(x)

float

In [24]:
x = sas.symget('syserr', outtype=int())
x

0

In [25]:
type(x)

int

## support for running on Databricks: display=['jupyter' | 'zeppelin' | 'databricks']

### display= is a Configuration Definition key, or can be specified on SASsession()
### this controls how HTML is rendered, which is different in each notebook server

# V3.1.8

## New versions of the submit() method: submitLOG() and submitLST()

## submitLOG()

In [26]:
# this runs the code and explicitly renders the LOG
sas.submitLOG("proc options option=memsize;run;")

47                                                         The SAS System                             09:42 Thursday, March 12, 2020

406        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
406      ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
407        
408        proc options option=memsize;run;

    SAS (r) Proprietary Software Release 9.4  TS1M5

 MEMSIZE=2147483648
                   Specifies the limit on the amount of virtual memory that can be used during a SAS session.
NOTE: PROCEDURE OPTIONS used (Total process time):
      real time           0.00 seconds
      cpu time            0.00 seconds
      

409        
410        ods html5 (id=saspy_internal) close;ods listing;
411        


## submitLST()

In [27]:
# this runs the code and explicitly renders the LST
sas.submitLST("proc print data=sashelp.cars(obs=2);run;")

Obs,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
1,Acura,MDX,SUV,Asia,All,"$36,945","$33,337",3.5,6,265,17,23,4451,106,189
2,Acura,RSX Type S 2dr,Sedan,Asia,Front,"$23,820","$21,761",2.0,4,200,24,31,2778,101,172


## But submitLST() has a method option too (listonly is the default): 
### method=['listonly' | 'listorlog' | 'listandlog' | 'logandlist']

In [28]:
sas.submitLST("proc print data=no_cars(obs=2);run;", method='listorlog')

In [29]:
sas.submitLST("proc print data=sashelp.cars(obs=2);run;", method='listandlog')

Obs,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
1,Acura,MDX,SUV,Asia,All,"$36,945","$33,337",3.5,6,265,17,23,4451,106,189
2,Acura,RSX Type S 2dr,Sedan,Asia,Front,"$23,820","$21,761",2.0,4,200,24,31,2778,101,172


In [30]:
sas.submitLST("proc print data=sashelp.cars(obs=2);run;", method='logandlist')

Obs,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
1,Acura,MDX,SUV,Asia,All,"$36,945","$33,337",3.5,6,265,17,23,4451,106,189
2,Acura,RSX Type S 2dr,Sedan,Asia,Front,"$23,820","$21,761",2.0,4,200,24,31,2778,101,172


# V3.1.9

## m5dsbug - an option added to workaround a bug in SAS until there is a hot fix

### the affects sd2df methods, for V9.4 M5 only, and only with multibyte character data

# V3.2.0

## User requested enhancements to df2sd() and sd2df()

## df2sd(..., datetimes={}, outfmts={})
### datatimes= allows for creating SAS date ot SAS time variables from a pands datetime64 column
### outfmts= allow you to specify the formats to assign to variable of the new data set

In [31]:
import pandas as pd
import saspy
import datetime
rows = [[datetime.datetime(1965, 1, 1, 8, 0, 1), datetime.datetime(1965, 1, 1, 8, 0, 1), datetime.datetime(1965, 1, 1, 8, 0, 1)], 
        [datetime.datetime(1966, 1, 1, 7, 0, 2), datetime.datetime(1966, 1, 1, 7, 0, 2), datetime.datetime(1966, 1, 1, 7, 0, 2)], 
        [datetime.datetime(1967, 1, 1, 6, 0, 3), datetime.datetime(1967, 1, 1, 6, 0, 3), datetime.datetime(1967, 1, 1, 6, 0, 3)], 
        [None, None, None], 
       ]
df = pd.DataFrame.from_records(rows, columns=['dt','d','t'])
df

,dt,d,t
0,1965-01-01 08:00:01,1965-01-01 08:00:01,1965-01-01 08:00:01
1,1966-01-01 07:00:02,1966-01-01 07:00:02,1966-01-01 07:00:02
2,1967-01-01 06:00:03,1967-01-01 06:00:03,1967-01-01 06:00:03
3,NaT,NaT,NaT


In [32]:
sd = sas.df2sd(df, datetimes={'d':'date', 't': 'time'}, outfmts={'d':'mmddyy10.'}, results='text')
sd.head()


                                                           The SAS System                         09:42 Thursday, March 12, 2020   2

                                    Obs    dt                                     d       t

                                     1     1965-01-01T08:00:01.000000    01/01/1965    08:00:01
                                     2     1966-01-01T07:00:02.000000    01/01/1966    07:00:02
                                     3     1967-01-01T06:00:03.000000    01/01/1967    06:00:03
                                     4                              .             .           .



In [33]:
sd.columnInfo()


                                                           The SAS System                         09:42 Thursday, March 12, 2020   3

                                                       The CONTENTS Procedure

                                            Alphabetic List of Variables and Attributes
 
                                            #    Variable    Type    Len    Format

                                            2    d           Num       8    MMDDYY10.  
                                            1    dt          Num       8    E8601DT26.6
                                            3    t           Num       8    E8601TM.   



## sd2df_[CSV | DISK] (..., dtypes={}, my_fmts=[False | True])
### dtypes= is the Pandas.read_csv() parameter to control the data types of the dataframe columns
### my_fmts= True causes use of formats on the dataset or specified via dsopts

In [34]:
rows = [[datetime.datetime(1965, 1, 1, 8, 0, 1), 10000.0, 1.30, 'a'], 
        [datetime.datetime(1966, 1, 1, 7, 0, 2), 20000.0, 2.30, 'b'], 
        [datetime.datetime(1967, 1, 1, 6, 0, 3), 30000.0, 3.30, ' '], 
        [datetime.datetime(1968, 1, 1, 5, 0, 4), 40000.0, 4.30,  ''], 
        [None,                                       5.0, 5.0,  'b'], 
        [None,                                       6.0, None, 'b'], 
       ]

df = pd.DataFrame.from_records(rows, columns=['dt','n1','n2', 's1'])
df
df.dtypes

dt    datetime64[ns]
n1           float64
n2           float64
s1            object
dtype: object

In [35]:
sd = sas.df2sd(df, 'v320', datetimes={'dt':'date'}, outfmts={'dt':'mmddyy10.', 'n1': 'comma32.2', 'n2': 'dollar32.2'}, results='text')
sd

Libref  = WORK
Table   = v320
Dsopts  = {}
Results = text

In [36]:
sd.columnInfo()


                                                           The SAS System                         09:42 Thursday, March 12, 2020   4

                                                       The CONTENTS Procedure

                                             Alphabetic List of Variables and Attributes
 
                                             #    Variable    Type    Len    Format

                                             1    dt          Num       8    MMDDYY10. 
                                             2    n1          Num       8    COMMA32.2 
                                             3    n2          Num       8    DOLLAR32.2
                                             4    s1          Char      1              



In [37]:
sd.head(10)


                                                           The SAS System                         09:42 Thursday, March 12, 2020   5

                  Obs            dt                                  n1                                  n2    s1

                   1     01/01/1965                           10,000.00                               $1.30    a 
                   2     01/01/1966                           20,000.00                               $2.30    b 
                   3     01/01/1967                           30,000.00                               $3.30      
                   4     01/01/1968                           40,000.00                               $4.30      
                   5              .                                5.00                               $5.00    b 
                   6              .                                6.00                                 .      b 



### all dataframe columns as strings

In [38]:
df2 = sd.to_df_DISK(dtype='str')
df2.dtypes

dt    object
n1    object
n2    object
s1    object
dtype: object

In [39]:
df2

,dt,n1,n2,s1
0,1965-01-01,10000,1.3,a
1,1966-01-01,20000,2.3,b
2,1967-01-01,30000,3.3,NaN
3,1968-01-01,40000,4.3,NaN
4,NaN,5,5,b
5,NaN,6,NaN,b


### all dataframe columns as stringsm but with the existing SAS formats applied 

In [40]:
df2 = sd.to_df_DISK(dtype='str', my_fmts=True)
df2.dtypes

dt    object
n1    object
n2    object
s1    object
dtype: object

In [41]:
df2

,dt,n1,n2,s1
0,01/01/1965,"10,000.00",$1.30,a
1,01/01/1966,"20,000.00",$2.30,b
2,01/01/1967,"30,000.00",$3.30,NaN
3,01/01/1968,"40,000.00",$4.30,NaN
4,NaN,5.00,$5.00,b
5,NaN,6.00,NaN,b


### make 'n2' be an int, and let the others be defaulted, with SAS formatting (override n2 format to numeric)

In [42]:
sd.dsopts = {'format':{'n1':'best32.'}}
df3 = sd.to_df_DISK(dtype={'n1':'int'}, my_fmts=True)
sd.dsopts = {}
df3

,dt,n1,n2,s1
0,01/01/1965,10000,$1.30,a
1,01/01/1966,20000,$2.30,b
2,01/01/1967,30000,$3.30,NaN
3,01/01/1968,40000,$4.30,NaN
4,NaN,5,$5.00,b
5,NaN,6,NaN,b


In [43]:
df3.dtypes

dt    object
n1     int32
n2    object
s1    object
dtype: object

In [44]:
df4 = sas.sd2df_DISK('v320', dtype={'n1':'int'}, my_fmts=True, dsopts = {'format':{'n1':'best32.'}})
df4

,dt,n1,n2,s1
0,01/01/1965,10000,$1.30,a
1,01/01/1966,20000,$2.30,b
2,01/01/1967,30000,$3.30,NaN
3,01/01/1968,40000,$4.30,NaN
4,NaN,5,$5.00,b
5,NaN,6,NaN,b


In [45]:
df4.dtypes

dt    object
n1     int32
n2    object
s1    object
dtype: object

# V3.2.1

## Continuing with enhancements to sd2df, add labels= for assigning labels to variables you create
### labels  - dict with column names and SAS Labels to assign to the new SAS data set

In [46]:
sd = sas.df2sd(df, 'v321', labels={'dt':'this is a date', 'n1': 'this has commas!', 'n2': 'this is money'}, 
               datetimes={'dt':'date'}, outfmts={'dt':'mmddyy10.', 'n1': 'comma32.2', 'n2': 'dollar32.2'}, results='text')
sd

Libref  = WORK
Table   = v321
Dsopts  = {}
Results = text

In [47]:
sd.columnInfo()


                                                           The SAS System                         09:42 Thursday, March 12, 2020   6

                                                       The CONTENTS Procedure

                                             Alphabetic List of Variables and Attributes
 
                                   #    Variable    Type    Len    Format        Label

                                   1    dt          Num       8    MMDDYY10.     this is a date  
                                   2    n1          Num       8    COMMA32.2     this has commas!
                                   3    n2          Num       8    DOLLAR32.2    this is money   
                                   4    s1          Char      1                                  



In [48]:
sd.head()


                                                           The SAS System                         09:42 Thursday, March 12, 2020   7

                  Obs            dt                                  n1                                  n2    s1

                   1     01/01/1965                           10,000.00                               $1.30    a 
                   2     01/01/1966                           20,000.00                               $2.30    b 
                   3     01/01/1967                           30,000.00                               $3.30      
                   4     01/01/1968                           40,000.00                               $4.30      
                   5              .                                5.00                               $5.00    b 



## Rename a SAS Data Set  SASdata.rename('new_name')

In [49]:
sd

Libref  = WORK
Table   = v321
Dsopts  = {}
Results = text

In [50]:
for dataset in sas.list_tables('work'):
    if dataset in [('V321', 'DATA'),('TOM1', 'DATA')]:
        print(dataset[0]+'.'+dataset[1])

V321.DATA


In [51]:
sd.rename('tom1')
sd

106                                                        The SAS System                             09:42 Thursday, March 12, 2020

953        
954        proc datasets dd=WORK nolist;
955        change 'v321'n = 'tom1'n;
956        run;

NOTE: Changing the name WORK.V321 to WORK.TOM1 (memtype=DATA).
956      !     quit;

NOTE: PROCEDURE DATASETS used (Total process time):
      real time           0.00 seconds
      cpu time            0.01 seconds
      

957        
958        


Libref  = WORK
Table   = tom1
Dsopts  = {}
Results = text

In [52]:
for dataset in sas.list_tables('work'):
    if dataset in [('V321', 'DATA'),('TOM1', 'DATA')]:
        print(dataset[0]+'.'+dataset[1])

TOM1.DATA


## Modify a Data Set, setting formats, informats, labels or changing the variable names    SASdata.modify(...)
<pre>   :param formats: dict of variable names and formats to assign
   :param informats: dict of variable names and informats to assign
   :param label: string of the label to assign to the data set; if it requires outer quotes, provide them
   :param renamevars: dict of variable names and new names tr rename the variables
   :param labelvars: dict of variable names and labels to assign to them
</pre>

In [53]:
sd.modify(formats={'dt':'B8601DT26.6'}, informats={'dt':'B8601DT26.6'}, label='data set label goes here',
                   renamevars={'s1':'Better Name'}, labelvars={'n1':'Commas are good'})

111                                                        The SAS System                             09:42 Thursday, March 12, 2020

1010       
1011       proc datasets dd=WORK nolist; modify 'tom1'n (label=data set label goes here);
1012       format 'dt'n B8601DT26.6;
1013       informat 'dt'n B8601DT26.6;
1014       rename 's1'n = 'Better Name'n;
NOTE: Renaming variable s1 to 'Better Name'n.
1015       label 'n1'n = Commas are good;
1016       ;run;

NOTE: MODIFY was successful for WORK.TOM1.DATA.
1016     !      quit;

NOTE: PROCEDURE DATASETS used (Total process time):
      real time           0.00 seconds
      cpu time            0.00 seconds
      

1017       
1018       


In [54]:
sd.contents()


                                                           The SAS System                         09:42 Thursday, March 12, 2020   8

                                                       The CONTENTS Procedure

                            Data Set Name        WORK.TOM1                     Observations          6 
                            Member Type          DATA                          Variables             4 
                            Engine               V9                            Indexes               0 
                            Created              03/12/2020 09:42:39           Observation Length    32
                            Last Modified        03/12/2020 09:42:40           Deleted Observations  0 
                            Protection                                         Compressed            NO
                            Data Set Type                                      Sorted                NO
                            Label                data set 